In [ ]:
%load_ext autoreload
%autoreload 2

# Graph Debugger Agent

In this example notebook we are going to demonstrate how Blar's ```GraphTraversalAgent``` can be used to create a test suitcase. We'll download a mock repo from Github, use ```GraphConstructor``` to build the graph from it, and upload it to Neo4j.

This is an introductory example of what can be achieved with Blar, but we are sure that has several other amazing use cases.

**NOTE:** Currently, this pack is configured to only work with `OpenAI` LLMs and `Neo4j` database. But feel free to copy/download the source code and edit as needed!

## Installation and Import

### Core dependencies

In [ ]:
%%pip install blar-graph --quiet --upgrade
%pip install python-dotenv --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import uuid
from blar_graph.graph_construction.core.graph_builder import GraphConstructor
from blar_graph.db_managers import Neo4jManager
from blar_graph.agents_tools.agents_examples.unit_test import get_unit_test_agent
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
import os

os.environ["NEO4J_URI"] = "neo4j+s://YOUR_NEO4J.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "YOUR_NEO4J_PASSWORD"
os.environ["OPENAI_API_KEY"] = "sk-..."

### Download the example repo


We will download an example repo with mock code to show how this agent works.

In [ ]:
%git clone https://github.com/blarApp/blar-example-repos.git

### Visualization dependencies

In [ ]:
%pip install yfiles_jupyter_graphs --quiet
%pip install neo4j graphdatascience --quiet

If running in Gooogle Colab run the following

In [ ]:
try:
    from google.colab import output

    output.enable_custom_widget_manager()
except:
    print("Error")
    pass

## Create the Graph

We initialize the `Neo4jManager` and pass it to the `GraphConstructor`. Then we create the example repo's graph by calling build_graph method. This method requires the path to the initial directory and the language of the code.

The build_graph method will scan the directory and recursively traverse the directories creating the nodes and relationships between them.

**NOTE:** Only python is supported at the moment


### Create unique Repo id

First we need to create a unique Id for the Repo. This is in case there is more than 1 repo in the same Neo4j DB. This way we can always query the repo we are interested in and not other repos.

In this case we will use uuid to generate a unique repoId. This repoId is later used to initialize the Neo4jManager.

In [ ]:
repoId = str(uuid.uuid4())
entityId = str(uuid.uuid4())

### Create the graph

In [ ]:
import traceback

graph_manager = Neo4jManager(repoId, entityId)

try:
    graph_constructor = GraphConstructor(graph_manager, entityId)
    nodes, relationships = graph_constructor.build_graph("blarApp/blar-example-repos")
    graph_manager.save_graph(nodes, relationships)
    graph_manager.close()
except Exception as e:
    print(e)
    print(traceback.format_exc())
    graph_manager.close()

Processed blar-example-repos/unit_test_agent/average_calculator.pyCreated 8 nodes
Created 11 edges


### Visualize Graph

For visualization we'll use the [yFiles](https://www.yworks.com/products/yfiles) library. This is an awesome interactive library that helps you visualize and explore graph data. Big shoutout to them!

In [ ]:
graph = graph_manager.get_whole_graph(result_format="graph")

from yfiles_jupyter_graphs import GraphWidget

w = GraphWidget(graph=graph)


def custom_edge_label_mapping(edge):
    """let the label be the negated index"""
    return ""


w.set_edge_label_mapping(custom_edge_label_mapping)

Here you can see the graph generated from the example repo. It has one directory, two files, and a few functions. The functions are simple math operations to show how the agent operates. You can go to the [repo](https://github.com/blarApp/blar-example-repos) to check the code yourself.

In [ ]:
w.show()

GraphWidget(layout=Layout(height='500px', width='100%'))

## Unit Test

In this section we will be utilizing the Blar unit test agent to create a unit test for the file average_calculator.py in the example repo.

The file looks like this:

```python 

from math_operations import add, divide


def calculate_average(numbers):
    """Calculate the average of a list of numbers."""
    if not numbers:
        raise ValueError("The list of numbers cannot be empty.")

    sum_of_numbers = 0
    for number in numbers:
        sum_of_numbers = add(sum_of_numbers, number)

    average = divide(sum_of_numbers, len(numbers))
    return average

```

This function imports two other functions that are crucial to understand the code to write accurate tests

We initialize the agent with our db manager. This is the information source our agent will use to traverse the graph.

In [ ]:
agent = get_unit_test_agent(graph_manager)

We describe the task and give the path of the file to process and create tests.

In [ ]:
list(agent.stream({"input": "Generate the unit test for the file average_calculator"}))



> Entering new AgentExecutor chain...

Invoking: `keword_search` with `{'query': 'average_calculator'}`


current node code:
 # average_calculator.py

from math_operations import add, divide


def calculate_average(numbers):
    # Code replaced for brevity. See node_id 3e2d542a-4bf2-4512-83eb-19da6e79cec8 

 current node neighbours: [{'node_id': '7f9c96b7-8213-45c8-bb7f-f53a12eece95', 'function_name': 'add', 'labels': ['FUNCTION']}, {'node_id': '02c7d13d-7c16-49db-be9a-e54922aabe18', 'function_name': 'divide', 'labels': ['FUNCTION']}, {'node_id': '3e2d542a-4bf2-4512-83eb-19da6e79cec8', 'function_name': 'calculate_average', 'labels': ['FUNCTION']}]
Invoking: `keword_search` with `{'query': '3e2d542a-4bf2-4512-83eb-19da6e79cec8'}`


current node code:
 def calculate_average(numbers):
    """Calculate the average of a list of numbers."""
    if not numbers:
        raise ValueError("The list of numbers cannot be empty.")

    sum_of_numbers = 0
    for number in numbers:
        sum_of_

[{'actions': [OpenAIToolAgentAction(tool='keword_search', tool_input={'query': 'average_calculator'}, log="\nInvoking: `keword_search` with `{'query': 'average_calculator'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Z9oU5YwxkWvrhHus3sMBcTKB', 'function': {'arguments': '{"query":"average_calculator"}', 'name': 'keword_search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'})], tool_call_id='call_Z9oU5YwxkWvrhHus3sMBcTKB')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_Z9oU5YwxkWvrhHus3sMBcTKB', 'function': {'arguments': '{"query":"average_calculator"}', 'name': 'keword_search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'})]},
 {'steps': [AgentStep(action=OpenAIToolAgentAction(tool='keword_search', tool_input={'query': 'average_calculator'}, log="\nInvoking: `keword_search` with `{'query': 'average_calculator'}`\n\

As you can see the agent navigates the graph fetching all the code involved in the file average_calculator. Then proceeds to write all the test cases that things are appropriate to fully test the function calculate_average.

**Note**: Due to the generative nature of LLMs you may not get exactly the same results.